In [16]:

def get_results(endpoint_url, query):
        sparql = SPARQLWrapper(endpoint_url)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        return sparql.query().convert()


def get_dicitionary_of_outbreaks_to_countries(output_file, print_dict = True ):

    import pandas as pd
    from SPARQLWrapper import SPARQLWrapper, JSON

    endpoint_url = "https://query.wikidata.org/sparql"
    query = """SELECT ?item ?itemLabel ?country ?countryLabel ?alternative
    WHERE 
    {
      ?item wdt:P361 wd:Q83741704.
      ?item wdt:P17 ?country.
      OPTIONAL { ?country skos:altLabel ?alternative . FILTER (lang(?alternative) = "en") }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """
    
    results = get_results(endpoint_url, query)
    dictionary = pd.json_normalize(results["results"]["bindings"])
    
    dictionary = dictionary[['item.value','itemLabel.value',"country.value", "countryLabel.value", "alternative.value"]]
    dictionary.columns = ["outbreak_qid", "outbreak", "country_qid", "country", "other_names"]

    outbreak_qids = dictionary["outbreak_qid"]
    dictionary["outbreak_qid"] = [item.split("/")[4] for item in outbreak_qids]
    country_qids = dictionary["country_qid"]
    dictionary["country_qid"] = [item.split("/")[4] for item in country_qids]   
    
    if print_dict:
        print(dictionary)

    dictionary.to_csv(output_file)
    

In [17]:
output_file_name = "country_to_outbreak_dictionary.csv"

get_dicitionary_of_outbreaks_to_countries(output_file_name)

    outbreak_qid                                outbreak country_qid  \
0      Q83872291   2019–20 coronavirus outbreak in Japan         Q17   
1      Q83872291   2019–20 coronavirus outbreak in Japan         Q17   
2      Q83872291   2019–20 coronavirus outbreak in Japan         Q17   
3      Q83872291   2019–20 coronavirus outbreak in Japan         Q17   
4      Q87250695     2020 coronavirus pandemic in Poland         Q36   
..           ...                                     ...         ...   
937    Q87199320  2020 coronavirus outbreak in Palestine     Q219060   
938    Q87199320  2020 coronavirus outbreak in Palestine     Q219060   
939    Q87199320  2020 coronavirus outbreak in Palestine     Q219060   
940    Q87199320  2020 coronavirus outbreak in Palestine     Q219060   
941    Q87250930   2020 coronavirus outbreak in Cameroon   Q29406548   

                country            other_names  
0                 Japan                  Nihon  
1                 Japan             N